In [2]:
import recordlinkage
import pandas as pd
import re

In [6]:
afi = pd.read_csv('scrapers/AFI.csv')
loc = pd.read_csv('scrapers/loc.csv')
bfi = pd.read_csv('scrapers/scraping_bfi.csv')

In [86]:
indexer = recordlinkage.Index()
indexer.full()
candidate_links = indexer.index(loc, afi)

In [87]:
c = recordlinkage.Compare()

c.string('title', 'title', method='jarowinkler', threshold=0.85, label='title')
c.exact('year', 'year', label='year')

features = c.compute(candidate_links, loc, afi)
print(len(features))

331200


In [88]:
potential_duplicates = features[features.sum(axis=1) > 1]

In [89]:
duplicates_df = pd.DataFrame(index=potential_duplicates.index.get_level_values(0), columns=['title', 'year'])

for idx in duplicates_df.index:
    title = loc.loc[idx, 'title']
    year = loc.loc[idx, 'year']
    duplicates_df.loc[idx] = [title, year]

In [90]:
duplicates_df.drop_duplicates(inplace=True)
duplicates_df

,title,year
32,The Lord of the Rings: Fellowship of the Ring,2001
54,A Clockwork Orange,1971
92,Platoon,1986
139,Spartacus,1960
143,Titanic,1997
...,...,...
798,West Side Story,1961
804,Who’s Afraid of Virginia Woolf?,1966
808,The Wild Bunch,1969
817,The Wizard of Oz,1939


In [91]:
merged_lists = pd.merge(loc, afi, how='left', on=['title', 'year'])

In [94]:
for title in merged_lists['title']:
    if title in duplicates_df['title'].values:
        merged_lists = merged_lists[~merged_lists['title'].isin(duplicates_df['title'])]

In [95]:
merged_lists

,title,year
0,Attica,1974
1,The Ballad of Gregorio Cortez,1982
2,Behind Every Good Man,1967
3,Betty Tells Her Story,1972
4,Bush Mama,1979
...,...,...
821,Woodstock,1970
822,Wuthering Heights,1939
824,Young Frankenstein,1974
825,Young Mr. Lincoln,1939
